RNN方法

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint



Using TensorFlow backend.


In [2]:
train_data = pd.read_csv("train.csv")   #(159571, 8)


In [3]:
#train_data
#train_data.shape

In [4]:
test_data = pd.read_csv('test.csv')

In [5]:
# test_data
#test_data.shape

In [6]:
#train_data = train_data.sample(frac=1)  #这一步是把顺序打乱，如果想恢复成正常顺序：df = df.sample(frac=1).reset_indedx(drop=True)

记住下面的几种pandas操作

In [7]:
#train_data.loc[0:3,:]
#train_data["comment_text"]
#train_data.loc[0,:]["comment_text"]

删除comment是none的数据，但是这组数据里面没有事none的，可以比较处理前后的shape来判断是否有none  

In [8]:
n,_ = train_data.shape
for i in range(n):
    if train_data.loc[i,:]["comment_text"] == None:
        train_data.drop(i,axis=0,inplace=True)   
#如果想要把none的地方用别的东西替换，可以用train_data["comment_text"].fillna("*****")  *****是用来替换的的东西     

提取train所有的评论数据

In [9]:
comments_train = train_data["comment_text"].values  
#print(comments_train.shape)   #(159571,)

提取所有的labels

In [10]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
labels = train_data[classes].values
#print(labels.shape)     #(159571, 6)

提取test的所有评论

In [11]:
n_test,_ = test_data.shape
for i in range(n_test):
    if test_data.loc[i,:]["comment_text"] == None:
        test_data.drop(i,axis=0,inplace=True)
        
comments_test = test_data["comment_text"].values  

#print(comments_test.shape)      #(153164,)


进行one-hot编码，将文本转化成向量

In [12]:
dic_len = 20000   #编码的单词的数量的上限
tokenizer = text.Tokenizer(num_words=dic_len)  #分词器
tokenizer.fit_on_texts(list(comments_train))  #先将array转换成list，fit_on_texts里面是要用以训练的文本列表，之后把每个单词编码
list_tokenized_train = tokenizer.texts_to_sequences(comments_train)  #之后句子就会变成单词编码的序列
list_tokenized_test = tokenizer.texts_to_sequences(comments_test)

#最后生成的序列可以print一下list_tokenized_train或者list_tokenized_train[0]看看
#使用例子：http://blog.csdn.net/zzulp/article/details/76146947 

将所有的长序列转化成等长序列 

In [13]:
#如果提供了参数maxlen，nb_timesteps=maxlen，否则其值为最长序列的长度。其他短于该长度的序列都会在后部填充0以达到该长度
maxlen=2000
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)   
X_test  = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)   
#train和test的maxlen必须相同，所以必须要提前定义maxlen
#X_train = sequence.pad_sequences(list_tokenized_train)   
#X_test  = sequence.pad_sequences(list_tokenized_test)
#可以print一下所有的句子都被向量化并且等长

In [14]:
#len(X_train[0])
#len(X_test[0])

构建RNN模型

In [15]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Activation,BatchNormalization
#from keras.models import 


In [109]:
'''
#下面的这个模型的问题在于，Sequential()只能讲不同的layer叠加，但是在中间的时候无法对数据进行处理，而在RNN之后我们要选取最后一个cell的输出
RNN_model=Sequential()
RNN_model.add(Embedding(input_dim = dic_len, output_dim = 64, input_length = maxlen))
#input_dim是字典里面包含的单词数量，output_dim是embedding扩展的维度，input_length：当输入序列的长度固定时，该值为其长度 
RNN_model.add(Bidirectional(LSTM(100, return_sequences=True)))
RNN_model.add(Bidirectional(LSTM(100, return_sequences=True)))
#RNN_model.add(GlobalMaxPool1D())
RNN_model.add(Dropout(0.3))
RNN_model.add(Dense(50, activation="relu"))
RNN_model.add(Dropout(0.1))
RNN_model.add(Dense(6, activation="sigmoid"))

RNN_model.compile(loss='binary_crossentropy',metrics=['acc'],optimizer=Adam(0.005))
RNN_model.summary()
'''


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 2000, 64)          1280000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 2000, 200)         132000    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 2000, 200)         240800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 2000, 200)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 2000, 50)          10050     
_________________________________________________________________
dropout_6 (Dropout)          (None, 2000, 50)          0         
_________________________________________________________________
dense_6 (Dense)              (None, 2000, 6)           306       
Total para

In [25]:
def get_model():
    embed_size = 64
    inp = Input(shape=(maxlen, ))
    x = Embedding(dic_len, embed_size)(inp)    #(20000,128)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    #x = x[:, -1, :]
    #x = np.reshape((embed_size, embed_size))
    x = Dropout(0.2)(x)
    x = Dense(50, activation="relu")(x)   #全连接
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)   #全连接
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()

    return model

In [26]:
RNN_model = get_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 2000)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 2000, 64)          1280000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 2000, 200)         132000    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 2000, 200)         240800    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 200)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                10050     
__________

In [ ]:
RNN_model.fit(X_train, labels, batch_size=64, epochs=2, validation_split=0.1)

In [117]:
#RNN_model.fit(X_train, labels, batch_size=64, epochs=2, validation_split=0.1 )

ValueError: Error when checking target: expected dense_6 to have 3 dimensions, but got array with shape (159571, 6)